In [5]:
%matplotlib widget


In [8]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, MultiPoint, Polygon
from shapely.wkt import loads
from scipy.spatial.ckdtree import cKDTree
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc, sys, os
import glob
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils
import string
import json
sys.path.append("..\scripts")
import spatial_functions

In [7]:

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=100.)
    if np.isnan(d[0]):
        return None

    else:
        near_ind = i[0]
    


        return grid_dict[var][near_ind]

def in_bounds(coords, b):
    """
    Function return true if point is within raster bounds
    coords = 1d numpy array (x,y)
    b = raseter dataset bounds from rasterio
    """
    x,y = coords[0], coords[1]
    
    if np.all([x > b.left, x < b.right,
                     y > b.bottom, y < b.top]):
        return True
    else:
        return False

def load_interpretations(infile):
    interpretations = {}
    df = pd.read_csv(infile, index_col = 0)
    for index, row in df.iterrows():
        interpretations[index] = row.to_dict()
    return interpretations
        

In [9]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\DalyR_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


# bring in the rjmcmc data

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\garj_workshop2\DR_rjmcmc_pmaps.nc"

# Read in the data

rj_dat = netCDF4.Dataset(infile)

rj_coords = np.column_stack((rj_dat['x'][:],
                              rj_dat['y'][:]))

# Now we bring in the AEM grids

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)

In [24]:
mask = lci_dat['line_index'][:] == np.where(lci_dat['line'][:] == 100001)[0]
mask.shape

(124422,)

In [22]:
np.where(lci_dat['line'][:] == 100001)[0]

array([0], dtype=int64)

In [9]:
# Lets have a quick look inside the netcdf file
rj_dat

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    nlayers_min: 1
    nlayers_max: 25
    nsamples: 500000
    nchains: 8
    nburnin: 50000
    thinrate: 10
    vmin: -3.0
    vmax: 0.0
    pmin: 0.0
    pmax: 500.0
    value_parameterization: LOG10
    position_parameterization: LINEAR
    dimensions(sizes): point(1095), data(41), depth(100), value(100), layer(25), chain(8), convergence_sample(50010)
    variables(dimensions): float64 observations(point,data), float64 errors(point,data), float64 depth(point,depth), float64 value(point,value), uint32 layer(point,layer), uint32 log10conductivity_histogram(point,depth,value), uint32 interface_depth_histogram(point,depth), uint32 nlayers_histogram(point,layer), uint32 chain(point,chain), uint32 convergence_sample(point,convergence_sample), float32 temperature(point,chain,convergence_sample), uint32 nlayers(point,chain,convergence_sample), float32 misfit(point,chain,convergence_sample), float32 logpp

In [8]:
# Lets plot the distribution of the points. Clicking on one will return the point index

# This function stores the top of the conductor in the dataframe on a click
def on_map_click(event):
    global point_index
    if event.xdata != None:
        x_, y_ = event.xdata, event.ydata
        distances, indices = nearest_neighbours([x_,y_], rj_coords, max_distance = 5000.)
        point_index = indices[0]
        
x,y = rj_dat.variables['x'][:], rj_dat.variables['y'][:]



fig, ax = plt.subplots(1,1,figsize = (15,15))
plt.scatter(x,y, c='k', s=1.5)
cid=  fig.canvas.mpl_connect('button_press_event', on_map_click)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# We will write our interpretations into a python dictionary with the fiducial as the key

infile = r"C:\Temp\pmap_interp\rj_interp.csv"

interpretations = load_interpretations(infile)

df = pd.DataFrame(interpretations).transpose()

df

,easting,northing,layer_depth,layer_elevation,standard_deviation
1124803.0,742111.75,8458588.0,166.0,-121.17,15.287792
1125963.5,747129.81,8457582.0,182.0,-102.67,15.287792
1126151.0,741491.69,8457586.0,186.0,-134.48,8.493218
1126434.5,744252.31,8455591.0,234.0,-175.33,16.986436
1126572.0,747945.62,8455592.0,238.0,-164.46,6.794574
1127570.5,749429.44,8453597.0,182.0,-109.47,8.493218
1127708.0,745489.00,8453596.0,250.0,-198.30,6.794574
1127970.0,743055.06,8451591.0,222.0,-181.10,6.794574
1128320.0,750885.75,8451581.0,162.0,-86.37,8.493218
1129320.0,748551.06,8450624.0,250.0,-189.54,16.986436


In [22]:
# Inteprolate with some gaussian process

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern


length_scale = 5000.

X = df[['easting', 'northing']].values

y = df['layer_elevation'].values[:,np.newaxis]

y_std = df['standard_deviation'].values[:,np.newaxis]

kernel = Matern(length_scale=length_scale)

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

ymin, ymax, = 8420000, 8460000
xmin, xmax = 740000, 778000

x__, y__ = np.mgrid[xmin:xmax:500, ymax:ymin:-500]

grid_coords = np.column_stack([x__.ravel(), y__.ravel()])

gp.fit(X,y)

y_pred = gp.predict(grid_coords)

gp.fit(X,y_std)

y_std_pred = gp.predict(grid_coords)

In [56]:
# Now we want to mask out values that are outside of our area. For this we will create a convex hull with a buffer


points = [Point(pt) for pt in X]
mpt = MultiPoint(points)
poly = mpt.convex_hull.buffer(2000.)

# create a mask showing which points are within the 
# polygon
mask = np.array([Point(x).within(poly) for x in grid_coords])

assert mask.sum() != mask.shape[0]


In [57]:
elevation = y_pred.flatten()
elevation[~mask] = np.nan

uncert = y_std_pred.flatten()
uncert[~mask] = np.nan

plt.close()

elevation_grid = elevation.reshape((x__.shape[0], y__.shape[1])).T
uncertainty_grid = uncert.reshape((x__.shape[0], y__.shape[1])).T

fig, (ax1,ax2) = plt.subplots(1,2, figsize = (12,6))
extent = (xmin, xmax, ymin, ymax)

im1 = ax1.imshow(elevation_grid,extent = extent, vmin = -300, vmax = 0)

cax = fig.add_axes([0.4, 0.6, 0.01, 0.25])

fig.colorbar(im1, cax=cax)

im2 = ax2.imshow(uncertainty_grid, extent = extent,
                vmin = 0., vmax = 50.)

cax2 = fig.add_axes([0.85, 0.6, 0.01, 0.25])

fig.colorbar(im2, cax=cax2)

ax1.scatter(X[:,0], X[:,1], marker = 'o', c = y.flatten(), 
            vmin = -300, vmax = 0, edgecolors  = 'k')

ax2.scatter(X[:,0], X[:,1], marker = 'o', c = y_std.flatten(), 
            vmin = 0, vmax = 50, edgecolors  = 'k')


plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:

        
        
        

def extract_data(idx = 0):
    """
    FUnction for extracting all the AEM data from the netCDF files
    """

    freq = rj_dat['log10conductivity_histogram'][idx].data.astype(np.float)
    
    easting = np.float(rj_dat['x'][idx].data)
    northing = np.float(rj_dat['y'][idx].data)

    cond_pdf = freq / freq.sum(axis =1)[0]

    cond_pdf[cond_pdf == 0] = np.nan
    
    cp_freq = rj_dat["interface_depth_histogram"][idx].data.astype(np.float)
    
    cp_pdf = cp_freq / freq.sum(axis =1)[0]
    
    laybins = rj_dat['nlayers_histogram'][idx].data
    
    lay_prob = laybins / freq.sum(axis =1)[0]
    
    condmin, condmax = rj_dat.vmin, rj_dat.vmax 
    
    ncond_cells = rj_dat.dimensions['value'].size
    
    cond_cells = np.linspace(condmin, condmax, ncond_cells)
    
    pmin, pmax = rj_dat.pmin, rj_dat.pmax
    
    
    depth_cells = rj_dat['depth'][idx].data
    
    extent = [cond_cells.min(), cond_cells.max(), depth_cells.max(), depth_cells.min()]
    
    mean = np.power(10,rj_dat['mean_model'][idx].data)
    p10 = np.power(10,rj_dat['p10_model'][idx].data)
    p50 = np.power(10,rj_dat['p50_model'][idx].data)
    p90 = np.power(10,rj_dat['p90_model'][idx].data)
    
    distances, indices = nearest_neighbours([easting, northing], lci_coords, max_distance = 50.)
    
    point_ind_lci = indices[0]
    
    lci_cond = lci_dat['conductivity'][point_ind_lci].data
    lci_depth_top = lci_dat['layer_top_depth'][point_ind_lci].data
    
    lci_doi = lci_dat['depth_of_investigation'][point_ind_lci].data
    
    misfit = np.sqrt(rj_dat['misfit'][idx].data)
    
    burnin = rj_dat.nburnin
    nsamples = rj_dat.nsamples
    sample_no = rj_dat['convergence_sample'][idx].data
    nchains = rj_dat.nchains
    elevation = rj_dat['elevation'][idx]
    line = int(rj_dat['line'][idx])
    fiducial = float(rj_dat['fiducial'][idx])
    elevation = rj_dat['elevation'][idx]
    
    return {'conductivity_pdf': cond_pdf, "change_point_pdf": cp_pdf, "conductivity_extent": extent,
           'cond_p10': p10, 'cond_p50': p50, 'cond_p90': p90, 'cond_mean': mean, 'depth_cells': depth_cells,
           'nlayer_bins': laybins, 'nlayer_prob': lay_prob, 'nsamples': nsamples, 'ndata': rj_dat.dimensions['data'].size,
           "nchains": nchains, 'burnin': burnin, 'misfit': misfit, 'sample_no': sample_no, 'cond_cells': cond_cells, 'lci_cond': lci_cond,
           'lci_depth_top': lci_depth_top, 'lci_doi': lci_doi, 'line': line, 'northing': northing, 'easting': easting, 'fiducial': fiducial,
           'elevation': elevation}
    
def DR_plot(D, outfile = None):
    fig = plt.figure(figsize = (12,10))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '5in'

    ax1 = fig.add_axes([0.05, 0.35, 0.35, 0.62])
    ax2 = fig.add_axes([0.45, 0.35, 0.2, 0.62])
    ax3 = fig.add_axes([0.70, 0.52, 0.2, 0.2])
    ax4 = fig.add_axes([0.72, 0.32, 0.16, 0.16])
    ax5 = fig.add_axes([0.1, 0.18, 0.76, 0.05])
    ax6 = fig.add_axes([0.1, 0.05, 0.76, 0.13])
    ax7 = fig.add_axes([0.70, 0.78, 0.2, 0.2])
    cbar_ax1 = fig.add_axes([0.05, 0.29, 0.35, 0.01])
    cbar_ax2 = fig.add_axes([0.88, 0.05, 0.01, 0.2])
    cbar_ax3 = fig.add_axes([0.9, 0.52, 0.01, 0.2])
    
    panel_kwargs = [{'title': '',
                      'color': 'black',
                      'ylabel': 'data \n residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 500.,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': 2.,
                      'cmap': 'jet',
                      'ylabel': 'elevation \n (mAHD)',
                      'vertical_exaggeration': 1.0}]


    # Plot probability map
    
    im = ax1.imshow(D['conductivity_pdf'], extent = D['conductivity_extent'],
                    aspect = 'auto', cmap = 'rainbow')
    
    #  PLot the median, and percentile plots
    ax1.plot(np.log10(D['cond_p10']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p10')
    ax1.plot(np.log10(D['cond_p90']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p90')
    ax1.plot(np.log10(D['cond_p50']), D['depth_cells'], c = 'k',label = 'p50')
    ax1.plot(np.log10(D['cond_mean']), D['depth_cells'], c = 'grey',label = 'mean')
    
    ax1.set_xticklabels([round(10 ** float(x), 4) for x in ax1.get_xticks()])

    # for lci layered model we do some processing
    lci_expanded = np.zeros(shape=2 * len(D['lci_cond']) + 1,
                                 dtype=np.float)

    lci_expanded[1:] = np.repeat(D['lci_cond'], 2)

    depth_expanded = (np.max(D['lci_depth_top']) + 10) * np.ones(shape=len(lci_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(D['lci_depth_top'], 2)

    ax1.plot(np.log10(lci_expanded), depth_expanded, c = 'pink',
             linestyle = 'dashed', label = 'lci')
    ax1.plot(ax1.get_xlim(), [D['lci_doi'], D['lci_doi']], c = 'yellow',
             label = 'LCI doi')
    ax1.set_title('rj-MCMC probability map')
    ax1.set_ylabel('depth (mBGL)')
    ax1.set_xlabel('Conductivity (S/m)')
    ax1.grid(which = 'both')
    ax1.set_xlim(D['conductivity_extent'][0], D['conductivity_extent'][1] )
    
    ax1.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#100.,0)#

        
    ax1.legend(loc = 3)
    ax2.plot(D['change_point_pdf'], D['depth_cells'], label = 'P(change point)')
    ax2.set_ylim(ax2.get_ylim()[::-1])
    ax2.set_yticks(np.arange(0, 500, 20.))
    ax2.set_title('change point probability')
    ax2.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#(100.,0)

    ax2.legend()
    ax2.grid(which = 'both')
    

    sample = D['sample_no'][:]
    
    # Add the misfit
    for i in range(D['misfit'].shape[0]):
       
        misfits = D['misfit'][i]
        ax4.plot(sample, misfits/D['ndata'])

    ax4.plot([1, D['nsamples']], [1,1], 'k')
    ax4.plot([D['burnin'], D['burnin']],[0.1,1e4], 'k')
    ax4.set_xlim([1, D['nsamples']])
    ax4.set_ylim(0.1, 1e4)

    ax4.set_xscale('log')
    ax4.set_yscale('log')

    ax4.set_xlabel("sample #")
    ax4.set_ylabel("Normalised misfit")
    
    im3 = ax3.imshow(elevation_grid,extent = extent, vmin = -300, vmax = 0)
    ax3.scatter(df['easting'].values, df['northing'].values, marker = 'o',
                c = df['layer_elevation'].values, 
                vmin = -300, vmax = 0, edgecolors  = 'k')
    ax3.plot(D['easting'],D['northing'],  'x', c = 'red')

    # conductivity plot
    
    ax7.imshow(np.log10(cond[9]), extent = [cond_dataset.bounds[0],
                                  cond_dataset.bounds[2],
                                  cond_dataset.bounds[1], 
                                  cond_dataset.bounds[3]],
              cmap = 'jet',
              vmin = np.log10(panel_kwargs[1]['vmin']*1000.),
              vmax = np.log10(panel_kwargs[1]['vmax']*1000.))
    
    ax7.set_xlim(D['easting'] - 10000., D['easting'] + 10000.)
    ax7.set_ylim(D['northing'] - 10000., D['northing'] + 10000.)
    ax7.plot(D['easting'],D['northing'],  'x', c = 'k')
    
    p1 = [gridded_vars[line]['easting'][0], gridded_vars[line]['easting'][-1]]
    p2 = [gridded_vars[line]['northing'][0], gridded_vars[line]['northing'][-1]]
    ax7.plot(p1, p2, 'k', linewidth = 0.5)
    ax7.set_title('LCI depth slice 61.8-71.6 mBGL', fontsize=10)
    ax7.tick_params(axis='both', which='major', labelsize=8)
    ax7.tick_params(axis='both', which='minor', labelsize=8)
    cb1 = fig.colorbar(im, cax=cbar_ax1, orientation='horizontal')
    cb1.set_label('probabilitiy', fontsize=10)

    res1 = plot_utils.plot_single_line(ax5, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax5.set_title('LCI conductivity section - ' + str(line))

    im2 = plot_utils.plot_grid(ax6, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])

    ax6.plot([dist, dist], [-500, 500], 'pink')
    ax6.set_xlabel("Distance along line (m)")
    
        
    cb2 = fig.colorbar(im2, cax=cbar_ax2, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)
    
    cb3 =  fig.colorbar(im3, cax=cbar_ax3, orientation='vertical')
    cb3.set_label('surface elevation mAHD')
    
    #ax5.set_xlim(dist - 5000.,
    #             dist + 5000.)
    ax6.set_xlim(dist - 5000., 
                 dist + 5000.)

    ax_array = np.array([ax1, ax2, ax3, ax4, ax5, ax6, ax7])
    
    return fig, ax_array   

In [69]:


# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 10., 2.

# We will use the lines from the rj

lines = rj_dat['line'][:]

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\lci\hdf5"

if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)

gridded_vars ={}

for line in lines:
    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    
    if os.path.exists(infile):
        f = h5py.File(infile, 'r')
        gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    else:
        gridded_vars[line] = plots.grid_variables(xres = xres, yres =yres, lines=[line],
                                    resampling_method = 'cubic', save_hdf5 = True,
                                    return_dict = True, hdf5_dir = hdf5_dir)
    f = None
    gc.collect()

In [70]:
# Now we bring in the Oolloo Jinduckin contact to plot

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\LeapFrog_export\OllooJind_contact.grd"

contact_dataset = rasterio.open(inRaster)

contact_elev = contact_dataset.read(1)

contact_elev[contact_elev == contact_dataset.get_nodatavals()] = np.nan


In [71]:
interpretations = {}

In [72]:
# This function stores the top of the conductor in the dataframe on a click
outdir = r"C:\temp\pmap_interp"

if not os.path.exists(outdir):
    os.mkdir(outdir)

outfile = os.path.join(outdir, "rj_interp.csv")    

# This function finds the width of the porbability interval that is >0.5
# times the local max probability
def full_width_half_max(max_idx, fmax):
    
    idx_upper = None
    idx_lower = None
    
    # positive direction
    for idx in np.arange(max_idx, D['depth_cells'].shape[0]):
        if D['change_point_pdf'][idx] <= fmax/2.:
            idx_upper = idx
            break
    # negative direction
    for idx in np.arange(max_idx, -1, -1):
        if D['change_point_pdf'][idx] <= fmax/2.:
            idx_lower = idx
            break
    # Now calculate the width
    if np.logical_and(idx_upper is not None, idx_lower is not None):
        return D['depth_cells'][idx_upper] - D['depth_cells'][idx_lower]
    else:
        return None
        
    

# Function for snapping to a layer point probability maximum
# from a click
def click2estimate(yclick):
    snap_window = 16
    ymin = yclick - snap_window/2
    ymax = yclick + snap_window/2
    
    # Get the change point probability array for the snap window interval
    
    idx = np.where(np.logical_and(D['depth_cells']>ymin, D['depth_cells']<ymax))
    
    # Now find the maximum cpp from this range
    idx_max = np.argmax(D['change_point_pdf'][idx]) + np.min(idx)
    fmax = D['change_point_pdf'][idx_max]
    interpreted_depth = D['depth_cells'][idx_max]
    
    # from https://en.wikipedia.org/wiki/Full_width_at_half_maximum
    fwhm = full_width_half_max(idx_max, fmax)
    
    if fwhm is not None:
        stdev = fwhm/(2*np.sqrt(2*np.log(2)))
    else:
        stdev = 50.
    return interpreted_depth, stdev

    


def pmap_click(event):
    if event.xdata != None and event.ydata != None:
        #We will use fiducial as a key
        fid = D['fiducial']
        depth, stdev = click2estimate(event.ydata)
        interpretations[fid] = {'easting': D['easting'],
                               'northing':  D['northing'],
                               'layer_depth': np.round(depth,0),
                               'layer_elevation': np.round(D['elevation'] - event.ydata,2),
                                'standard_deviation': np.round(stdev,0)
                               }
        # Save the interpretation
        pd.DataFrame(interpretations).transpose().to_csv(outfile)


In [78]:
point_index = 233

In [86]:
plt.close('all')


D = extract_data(point_index)

line = np.int(rj_dat['line'][point_index].data)

# Find distance along the lci section
dist = xy_2_var(gridded_vars[line],
                 np.array([[D['easting'], D['northing']]]),
                 'grid_distances')


point_ind_lci= dist

fig, ax_array = DR_plot(D)

point_index += 5 

plt.show()

cid=  fig.canvas.mpl_connect('button_press_event', pmap_click)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:178: RuntimeWarning: invalid value encountered in log10
